#Reglas del Juego:
1. El tablero será de tamaño 10x10.
2. Cada jugador tendrá 5 barcos de diferentes tamaños:
- portaaviones (tamaño 5)
- acorazado (tamaño 4)
- crucero (tamaño 3)
- submarino (tamaño 3)
- destructor (tamaño 2)
3. Los barcos podrán colocarse tanto en posición horizontal como vertical.
4. Durante el turno de un jugador, este deberá elegir una coordenada para atacar (fila y columna). Si acierta, el barco atacado recibirá un golpe. Si se destruyen todas las partes de un barco, este se considerará hundido.
5. El juego termina cuando un jugador logra hundir todos los barcos del oponente.

In [1]:
print("Bienvenido al juego de Hundir la Flota")

Bienvenido al juego de Hundir la Flota


In [1]:
class Barco:
    def __init__(self, nombre, tamaño,posicion,golpes):
        self.nombre=nombre
        self.tamaño=tamaño
        self.posicion=posicion
        self.golpes=golpes


    def recibir_golpe(self):
        self.golpes+=1
        if self.golpes==self.tamaño:
            return True

    def esta_hundido(self):
        return self.golpes==self.tamaño

In [ ]:
class Tablero(Barco):
    def __init__(self, tablero,barcos:Barco):
        self.tablero=tablero
        self.barcos=barcos

    def colocar_barco(self,orientacion:str):
        if orientacion=="horizontal":
            for i in range(self.barcos.tamaño):
                self.barcos.nombre=self.tablero[self.barcos.posicion[0]][self.barcos.posicion[1]+i]
        else:
            for i in range(self.barcos.tamaño):
                self.barcos.nombre=self.tablero[self.barcos.posicion[0]+i][self.barcos.posicion[1]]
        


    def recibir_ataque(self,fila,columna):
        for barco in self.barcos:
            if self.barcos.nombre==self.tablero[fila][columna]:
                if self.barcos.recibir_golpe():
                    print("Hundido")
                    return True
                else:
                    print("Tocado")
                    return True
        else:
            print("Agua")
            return False


    def mostrar_tablero(self):
        for fila in self.tablero:
            print(" ".join(fila))
            print("\n")
        


    def todos_barcos_hundidos(self):
        for barco in self.barcos:
            if not barco.esta_hundido():
                return False
            return True
        

In [3]:
class Jugador(Tablero):
    def __init__(self, nombre, tablero:Tablero):
        self.nombre=nombre
        self.tablero=tablero

    def atacar(self,oponente, fila, columna):
        if oponente.tablero.recibir_ataque(fila,columna):
            print("Tocado")
            return True
        elif oponente.tablero.todos_barcos_hundidos():
            print("Hundido")
            return True
        elif oponente.tablero.todos_barcos_hundidos():
            print("Enhorabuena, has ganado")
            return True
        else:
            print("Agua")
            return False

        

In [4]:
class Juego:
    def iniciar_juego(self):
        print("Bienvenido al juego de Hundir la Flota")
        nombre_jugador1 = input("Ingrese el nombre del jugador 1: ")
        nombre_jugador2 = 'Computadora'
        return Jugador(nombre_jugador1, Tablero(10,10)), Jugador(nombre_jugador2, Tablero(10,10))

    def turno(self, jugador):
        print(f"Turno de {jugador.nombre}")
        fila = int(input("Ingrese la fila de ataque: "))
        columna = int(input("Ingrese la columna de ataque: "))
        return jugador.atacar(jugador, fila, columna)

In [ ]:
# portaviones=Barco("Portaaviones",5,(int(input('introduzca la fila:')),int(input('introduzca la columna'))),0)
# acorazado=Barco("Acorazado",4,(int(input('introduzca la fila:')),int(input('introduzca la columna'))),0)
# crucero=Barco("Crucero",3,(int(input('introduzca la fila:')),int(input('introduzca la columna'))),0)
# submarino=Barco("Submarino",3,(int(input('introduzca la fila:')),int(input('introduzca la columna'))),0)
# destructor=Barco("Destructor",2,(int(input('introduzca la fila:')),int(input('introduzca la columna'))),0)

In [23]:
import numpy as np

In [ ]:
u = np.array([2, 5])


[2 5]


In [ ]:
tablero_jugador=

In [13]:
jugador_1=Jugador(input('introduzca su nombre:'),tablero_jugador)


In [16]:
barco_1=Barco("Portaaviones",5,(4,0),0)

In [21]:
print(Tablero(10,10).mostrar_tablero())

TypeError: 'int' object is not iterable

In [19]:
tablero_jugador.mostrar_tablero()

TypeError: 'int' object is not iterable

In [ ]:
barco_1.

TypeError: Tablero.colocar_barco() missing 1 required positional argument: 'orientacion'

In [9]:

Tablero.colocar_barco('vertical')



TypeError: Tablero.colocar_barco() missing 1 required positional argument: 'orientacion'

In [29]:
Tablero.colocar_barco(Barco("Portaaviones",5,(4,0),0),'horizontal')
Tablero.mostrar_tablero

TypeError: Tablero.colocar_barco() missing 1 required positional argument: 'orientacion'

In [24]:


for i in range(1,6):
    palabra=input(f'Introduzca el nombre del barco {i}:')
    if palabra=='portaviones':
        jugador_1.tablero.colocar_barco(Barco("Portaaviones",5,(int(input('introduzca la fila:')),int(input('introduzca la columna'))),0),str(input('horizontal o vertical:')))
    elif palabra=='acorazado':
        jugador_1.tablero.colocar_barco(Barco("Acorazado",4,(int(input('introduzca la fila:')),int(input('introduzca la columna'))),0))
    elif palabra=='crucero':
        jugador_1.tablero.colocar_barco(Barco("Crucero",3,(int(input('introduzca la fila:')),int(input('introduzca la columna'))),0))
    elif palabra=='submarino':
        jugador_1.tablero.colocar_barco(Barco("Submarino",3,(int(input('introduzca la fila:')),int(input('introduzca la columna'))),0))
    elif palabra =='destructor':
        jugador_1.tablero.colocar_barco(Barco("Destructor",2,(int(input('introduzca la fila:')),int(input('introduzca la columna'))),0))
        
    i+=1
    
print('Tablero Jugador 1:',Tablero.mostrar_tablero())


TypeError: Tablero.colocar_barco() missing 1 required positional argument: 'orientacion'